In [68]:
import os
import traceback
import logging
import os.path

In [53]:
# simversion = '_v01'
simversion = '_plasmid_v01'
# simversion = '_v00'
prepare_database_4_multicsar = 0
run_art = 0 # 1 True, 0 False
run_spades = 0
run_panta = 0
run_multicsar = 0
run_ragout = 0
run_pangraph = 0
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kp100random/' # for pangraph, Ragout, multi-CSAR #v3
pangenome_data = '/data/hoan/amromics/data/ncbi/Ecoli100random/'
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kptest/' # for pangraph, Ragout, multi-CSAR
simversion = '_'+ pangenome_data.split('/')[-2] + simversion
quast_output = '/data/hoan/amromics/genome-graph/scaffold_output/quastResults' + simversion
scaffold_out_dir = '/data/hoan/amromics/genome-graph/scaffold_output/'

In [54]:
print(simversion)

_Ecoli100random_plasmid_v01


In [55]:
if prepare_database_4_multicsar==1:
    os.system('rm -r ' + pangenome_data + 'fna/')
    os.system('mkdir ' + pangenome_data + 'fna/')
    os.system('cp -r '+ pangenome_data + '*.fna.gz '+ pangenome_data + 'fna/')
    os.system('~/miniconda3/envs/panta/bin/gzip -d ' + pangenome_data + 'fna/*.fna.gz')

### Simualate reads using ART

In [56]:
# https://github.com/scchess/Art/blob/master/art_illumina_README

In [57]:
sim_dir = '/data/hoan/amromics/simulation/'
art_bin = '/data/hoan/amromics/simulation/art_bin/./art_illumina'

In [58]:
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fasta'
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fna'
ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna'
sim_output = '/data/hoan/amromics/simulation/art_output/paired_dat' + simversion
# sim_output = '/data/hoan/amromics/simulation/art_output/paired_dat_test'

In [59]:
if run_art:
    # os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 60 -o '+sim_output) # _v3
    # os.system(art_bin+' -ss HS25 -sam -i '+ref_data+' -p -l 150 -f 70 -m 400 -s 60 -o '+sim_output) # _v0
    os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 60 -o '+sim_output) # _v01

### Run SPADES assembly

In [60]:
spades_output = '/data/hoan/amromics/simulation/art_output/' + 'spades_output' + simversion
spades_output

'/data/hoan/amromics/simulation/art_output/spades_output_Ecoli100random_plasmid_v01'

In [61]:
if run_spades:
    spades_bin ='~/miniconda3/envs/amromics/bin/spades.py'
    os.system(spades_bin+' --isolate -1 '+sim_output+'1.fq -2 '+sim_output+'2.fq -t 50 -o '+spades_output)

### Run panta

In [62]:
# move spades ouput to panta dir
print('Pangenome data: ', pangenome_data)
if run_spades:
    os.system('cp '+spades_output+'/contigs.fasta ' + pangenome_data + 'g1.fna')
    os.system('rm ' + pangenome_data + 'g1.fna.gz')
    os.system('~/miniconda3/envs/panta/bin/gzip -k ' + pangenome_data + 'g1.fna')

Pangenome data:  /data/hoan/amromics/data/ncbi/Ecoli100random/


In [63]:
# Run panta (change output dir)
# https://askubuntu.com/questions/1252439/not-able-to-activate-conda-environment-through-os-system-command-in-python
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate panta && '
panta_bin =conda_dir+'python /data/hoan/amromics/panta/panta.py'
panta_output = '/data/hoan/amromics/panta/examples/test/output' + simversion
# panta_output = '/data/hoan/amromics/panta/examples/test/output_Kp100' 
print('PANTA OUTPUT: ', panta_output)
if run_panta:
    cmd_panta = panta_bin + ' -p init -a ' +pangenome_data+ '*.fna.gz -o '+panta_output +' -as -s'
    os.system(cmd_panta)
else:
    print("Please RUN PANTA in bash file, here DONOT WORK Because of abPOA")

PANTA OUTPUT:  /data/hoan/amromics/panta/examples/test/output_Ecoli100random_plasmid_v01
Please RUN PANTA in bash file, here DONOT WORK Because of abPOA


### Run pangraph

In [38]:
try:
    %load_ext autoreload
    %autoreload 2
except Exception as e:
    logging.error(traceback.format_exc())
from pangraph import PanGraph

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# set parameters
data_dir = panta_output 
incomplete_sample_name = 'g1'
assem_dir = spades_output
fasta_gen = 'partial' # 'all', 'partial'

In [40]:
pangraph_output_greedy = scaffold_out_dir + 'pangraph_greedy'+ simversion + '.fasta'
pangraph_output_opt = scaffold_out_dir + 'pangraph_opt'+ simversion + '.fasta'

In [41]:
if run_pangraph:
    # https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
    pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)
    try:
        maximum_matching = 'greedy'
        pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_greedy, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

Should we use target sequence as reference, hence, the strand will be ok
Set minimum on number of nucleotides =  10 NUMBER OF COMPUTED CONTIGS: 462
Clip the matrixn 0.2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Use union graph: overlap + assembly
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Success! File written


In [42]:
if run_pangraph:
    try:
        maximum_matching = 'opt'
        pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

Compute maximum matching
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Will test this, Ok?
Success! File written


In [43]:
# try:
#     maximum_matching = 'greedy'
#     pangraph_output_greedy = spades_output + '/contigs_concat_'+ maximum_matching+ simversion + '.fasta'
#     pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
# except Exception as e:
#     logging.error(traceback.format_exc())
#     # Logs the error appropriately. 

### Run scaffold methods

In [64]:
### Multi-CSAR
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && '
csar_bin = conda_dir + '/data/hoan/amromics/assembly/Multi-CSAR/./multi-csar.php'

In [65]:
multicsar_output = scaffold_out_dir + 'multicsar' + simversion
if run_multicsar:
    os.system('rm ' + pangenome_data + 'fna/g1*')
    os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'fna/ --nuc -o '+multicsar_output)
    # os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'*.fna --nuc -o '+multicsar_output)

In [66]:
### Ragout
ragout_bin = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && ragout '
ragout_output = scaffold_out_dir + 'ragout' + simversion
if run_ragout:
    # write receipt file
    import glob
    ref_files_list = glob.glob(pangenome_data+'fna/*')
    n_files = len(ref_files_list)
    receipt_file_dir = '/data/hoan/amromics/assembly/Ragout/kp100'+simversion+'.rcp'
    f = open(receipt_file_dir, "w")
    f.write('.references = ')
    for idx in range(n_files-1):
        f.write('r' + str(idx) + ',')
    f.write('r'+str(n_files-1)+'\n')
    f.write(".target = mg1655\n\n")
    for idx in range(n_files):
        f.write('r'+str(idx)+'.fasta =' + ref_files_list[idx] + '\n')
    f.write('mg1655.fasta = ' + spades_output+'/contigs.fasta\n')
    f.close()

In [67]:
if run_ragout:
    os.system(ragout_bin + receipt_file_dir + ' --outdir ' + ragout_output + ' --refine')
    # ragout kp100_v2.rcp --outdir output_Kp100p_v2/ --refine

In [70]:
print(ragout_output)

/data/hoan/amromics/genome-graph/scaffold_output/ragout_Ecoli100random_plasmid_v01


### Quast 

In [49]:
quast_output

'/data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli100random_plasmid_v01'

In [69]:
quast_bin = 'python /data/hoan/amromics/spades_quast/quast-5.2.0/quast.py '
spades_output_fasta = spades_output+'/contigs.fasta'
multicsar_output_fasta = multicsar_output +'/multi-csar.nuc.out.fna'
ragout_output_fasta = ragout_output + '/mg1655_scaffolds.fasta'
if True:
    if os.path.isfile(ragout_output_fasta):
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+ragout_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, Ragout, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')
    else:
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')

/data/hoan/amromics/spades_quast/quast-5.2.0/quast.py /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli100random_plasmid_v01.fasta /data/hoan/amromics/genome-graph/scaffold_output/pangraph_greedy_Ecoli100random_plasmid_v01.fasta /data/hoan/amromics/genome-graph/scaffold_output/multicsar_Ecoli100random_plasmid_v01/multi-csar.nuc.out.fna /data/hoan/amromics/simulation/art_output/spades_output_Ecoli100random_plasmid_v01/contigs.fasta -l Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES -r /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna -o /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli100random_plasmid_v01 --silent --extensive-mis-size 5000 --local-mis-size 3000


System information:
  OS: Linux-3.10.0-1160.15.2.el7.x86_64-x86_64-with-redhat-7.9-Nitrogen (linux_64)
  Python version: 2.7.5
  CPUs number: 72

Started: 2022-12-17 06:13:34

Logging to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli100ran

Unable to parse the pattern
Unable to parse the pattern
Unable to parse the pattern


Done.

2022-12-17 06:13:39
Running Contig analyzer...
Done.

2022-12-17 06:13:42
Running NA-NGA calculation...
Done.

2022-12-17 06:13:43
Running Genome analyzer...
  NOTICE: No file with genomic features were provided. Use the --features option if you want to specify it.

  NOTICE: No file with operons were provided. Use the -O option if you want to specify it.
Done.

NOTICE: Genes are not predicted by default. Use --gene-finding or --glimmer option to enable it.

2022-12-17 06:13:45
Creating large visual summaries...
This may take a while: press Ctrl-C to skip this step..
  1 of 2: Creating PDF with all tables and plots...
  2 of 2: Creating Icarus viewers...
Done

2022-12-17 06:13:47
RESULTS:
  Text versions of total report are saved to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli100random_plasmid_v01/report.txt, report.tsv, and report.tex
  Text versions of transposed total report are saved to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecol

In [51]:
# # pangraph.H["weight"][0:4]
# iteration = 0
# for u,v,a in pangraph.H.edges(data=True):
#     print(a)
#     iteration += 1
#     if iteration > 10:
#         break;